# CW Laser Tuning Technique

In [ ]:
import Ring as rng
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import pickle

%load_ext autoreload
%autoreload 2

## Ring 1 - Forward tuning with no effects (Multiple-Soliton formation)

### Microresonator parameters

In [ ]:
parameters_ring1 =  {  
                        'N': 511, # Number of modes. It must be odd!
                        'n0': 2.4, # Refractive index
                        'n2': 2.4e-19, # Nonlinear reftactive index [m^2/W]
                        'FSR': 100e9, # Free Spectral Range [Hz]
                        'lambda0': 1553.4e-9, # CW pump wavelength [m]
                        'kappa': 3e8, # Optical linewidth [Hz]
                        'eta': 0.5, # Coupling efficiency
                        'Veff': 1e-15, # Effective mode volume [m^3]
                        'D2': 2.5e6, # Second order dispersion [Hz]
                        'Pin': 2 # Pump power [W]
                    }

In [ ]:
ring1 = rng.Ring(parameters_ring1) # Init Ring class

### Simulation parameters

In [ ]:
simulation_options_ring1 =  { 
                                'Effects': None, # None, "Thermal" or "Avoided mode crossings"
                                'Noise': False # True or False (White noise)
                            }

forward_parameters_ring1 =  {
                                'dseta_start': -10, # Normalized detuning start
                                'dseta_end': 45, # Normalized detuning end
                                'dseta_step': 0.01, # Tuning step
                                'roundtrips_step': 10, # Roundtrips per tuning step
                                'Amu0': np.random.randn(parameters_ring1['N']) + 1j * np.random.randn(parameters_ring1['N']), # Initial field
                            }

### Numerical simulation

In [ ]:
dseta_forward_ring1, amu_forward_ring1, _ = ring1.numerical_simulation(forward_parameters_ring1, simulation_options_ring1) # Run forward simulation

### Plot results

In [ ]:
interact(lambda dseta_snap: ring1.plot_results(dseta_forward_ring1, amu_forward_ring1, dseta_snap=dseta_snap), 
         dseta_snap=widgets.FloatSlider(min=forward_parameters_ring1['dseta_start'], max=forward_parameters_ring1['dseta_end'], step=forward_parameters_ring1['dseta_step'], value=0, continuous_update=False)); # Plot results from simulation

### Save data

In [ ]:
data_ring1 =    {
                    'parameters_ring': parameters_ring1,
                    'simulation_options': simulation_options_ring1,
                    'forward_parameters': forward_parameters_ring1,
                    'dseta_forward': dseta_forward_ring1,
                    'amu_forward': amu_forward_ring1
                }

In [ ]:
!mkdir -p data
with open('./data/data_ring1.pickle', 'wb') as f:
    pickle.dump(data_ring1, f, pickle.HIGHEST_PROTOCOL)

### Load data

In [ ]:
with open('./data/data_ring1.pickle', 'rb') as f:
    data = pickle.load(f)
    parameters_ring1 = data["parameters_ring"]
    simulation_options_ring1 = data["simulation_options"]
    forward_parameters_ring1 = data["forward_parameters"]
    dseta_forward_ring1 = data["dseta_forward"]
    amu_forward_ring1 = data["amu_forward"]

ring1 = rng.Ring(parameters_ring1) # Init Ring class

In [ ]:
interact(lambda dseta_snap: ring1.plot_results(dseta_forward_ring1, amu_forward_ring1, dseta_snap=dseta_snap), 
         dseta_snap=widgets.FloatSlider(min=forward_parameters_ring1['dseta_start'], max=forward_parameters_ring1['dseta_end'], step=forward_parameters_ring1['dseta_step'], value=0, continuous_update=False, font_size=75)); # Plot results from simulation

## Ring 2 - Forward and backward tuning with thermal effects (Soliton switching)

### Microresonator parameters

In [ ]:
parameters_ring2 =  {  
                        'N': 511, # Number of modes. It must be odd!
                        'n0': 2.4, # Refractive index
                        'n2': 2.4e-19, # Nonlinear reftactive index [m^2/W]
                        'FSR': 100e9, # Free Spectral Range [Hz]
                        'lambda0': 1553.4e-9, # CW pump wavelength [m]
                        'kappa': 3e8, # Optical linewidth [Hz]
                        'eta': 0.5, # Coupling efficiency
                        'Veff': 1e-15, # Effective mode volume [m^3]
                        'D2': 2.5e6, # Second order dispersion [Hz]
                        'Pin': 2 # Pump power [W]
                    }

In [ ]:
ring2 = rng.Ring(parameters_ring2) # Init Ring class

### Simulation parameters

In [ ]:
simulation_options_ring2 =  { 
                                'Effects': 'Thermal', # None, "Thermal" or "Avoided mode crossings"
                                'Noise': False # True or False (White noise)
                            }

forward_parameters_ring2 =  {
                                'dseta_start': -10, # Normalized detuning start
                                'dseta_end': 65, # Normalized detuning end
                                'dseta_step': 0.01, # Tuning step
                                'roundtrips_step': 10, # Roundtrips per tuning step
                                'Amu0': np.random.randn(parameters_ring2['N']) + 1j * np.random.randn(parameters_ring2['N']), # Initial field
                                'theta0': 0, # Initial normalized variation of temperature
                                'tauT': 1e-8, # Thermal relaxation time [s]
                                'n2T': 2.4e-18 # Coefficient of thermal nonlinearity [m^2/W]
                            }

backward_parameters_ring2 = {
                                'dseta_start': 65, # Normalized detuning start
                                'dseta_end': 10, # Normalized detuning end
                                'dseta_step': -0.001, # Tuning step
                                'roundtrips_step': 20, # Roundtrips per tuning step
                                'tauT': 1e-12, # Thermal relaxation time [s]
                                'n2T': 2.4e-18 # Coefficient of thermal nonlinearity [m^2/W]
                            }

### Numerical simulation

In [ ]:
dseta_forward_ring2, amu_forward_ring2, theta_forward_ring2 = ring2.numerical_simulation(forward_parameters_ring2, simulation_options_ring2) # Run forward simulation

In [ ]:
dseta_backward_ring2, amu_backward_ring2, theta_backward_ring2 = ring2.numerical_simulation(backward_parameters_ring2, simulation_options_ring2, dseta_forward_ring2, amu_forward_ring2, theta_forward_ring2) # Run backward simulation

### Plot results

In [ ]:
interact(lambda dseta_snap: ring2.plot_results(dseta_forward_ring2, amu_forward_ring2, dseta_backward_ring2, amu_backward_ring2, dseta_snap=dseta_snap), 
         dseta_snap=widgets.FloatSlider(min=forward_parameters_ring2['dseta_start'], max=forward_parameters_ring2['dseta_end'], step=forward_parameters_ring2['dseta_step'], value=0, continuous_update=False)); # Plot results from simulation

### Save data

In [ ]:
data_ring2 =    {
                    'parameters_ring': parameters_ring2,
                    'simulation_options': simulation_options_ring2,
                    'forward_parameters': forward_parameters_ring2,
                    'backward_parameters': backward_parameters_ring2,
                    'dseta_forward': dseta_forward_ring2,
                    'amu_forward': amu_forward_ring2,
                    'theta_forward': theta_forward_ring2,
                    'dseta_backward': dseta_backward_ring2,
                    'amu_backward': amu_backward_ring2,
                    'theta_backward': theta_backward_ring2,
                }

In [ ]:
!mkdir -p data
with open('./data/data_ring2.pickle', 'wb') as f:
    pickle.dump(data_ring2, f, pickle.HIGHEST_PROTOCOL)

### Load data

In [ ]:
with open('./data/data_ring2.pickle', 'rb') as f:
    data = pickle.load(f)
    parameters_ring2 = data["parameters_ring"]
    simulation_options_ring2 = data["simulation_options"]
    forward_parameters_ring2 = data["forward_parameters"]
    backward_parameters_ring2 = data["backward_parameters"]
    dseta_forward_ring2 = data["dseta_forward"]
    amu_forward_ring2 = data["amu_forward"]
    theta_forward_ring2 = data["theta_forward"]
    dseta_backward_ring2 = data["dseta_backward"]
    amu_backward_ring2 = data["amu_backward"]
    theta_backward_ring2 = data["theta_backward"]

ring2 = rng.Ring(parameters_ring2) # Init Ring class

In [ ]:
interact(lambda dseta_snap: ring2.plot_results(dseta_forward_ring2, amu_forward_ring2, dseta_backward_ring2, amu_backward_ring2, dseta_snap=dseta_snap), 
         dseta_snap=widgets.FloatSlider(min=forward_parameters_ring2['dseta_start'], max=forward_parameters_ring2['dseta_end'], step=forward_parameters_ring2['dseta_step'], value=0, continuous_update=False)); # Plot results from simulation

## Ring 3 - Forward tuning with avoided mode crossings (Soliton crystal formation)

### Microresonator parameters

In [ ]:
parameters_ring3 =  {  
                        'N': 511, # Number of modes. It must be odd!
                        'n0': 2.4, # Refractive index
                        'n2': 2.4e-19, # Nonlinear reftactive index [m^2/W]
                        'FSR': 100e9, # Free Spectral Range [Hz]
                        'lambda0': 1553.4e-9, # CW pump wavelength [m]
                        'kappa': 3e8, # Optical linewidth [Hz]
                        'eta': 0.5, # Coupling efficiency
                        'Veff': 1e-15, # Effective mode volume [m^3]
                        'D2': 2.5e6, # Second order dispersion [Hz]
                        'Pin': 0.5 # Pump power [W]
                    }

In [ ]:
ring3 = rng.Ring(parameters_ring3) # Init Ring class

### Simulation parameters

In [ ]:
simulation_options_ring3 =  { 
                                'Effects': 'Avoided mode crossings', # None, "Thermal" or "Avoided mode crossings"
                                'Noise': False # True or False (White noise)
                            }

forward_parameters_ring3 =  {
                                'dseta_start': -10, # Normalized detuning start
                                'dseta_end': 12.5, # Normalized detuning end
                                'dseta_step': 0.01, # Tuning step
                                'roundtrips_step': 50, # Roundtrips per tuning step
                                'Amu0': np.random.randn(parameters_ring3['N']) + 1j * np.random.randn(parameters_ring3['N']), # Initial field
                                'mode_perturbated': 15 # Position of the modal crossing
                            }

### Numerical simulation

In [ ]:
dseta_forward_ring3, amu_forward_ring3, _ = ring3.numerical_simulation(forward_parameters_ring3, simulation_options_ring3) # Run forward simulation

### Plot results

In [ ]:
interact(lambda dseta_snap: ring3.plot_results(dseta_forward_ring3, amu_forward_ring3, dseta_snap=dseta_snap), 
         dseta_snap=widgets.FloatSlider(min=forward_parameters_ring3['dseta_start'], max=forward_parameters_ring3['dseta_end'], step=forward_parameters_ring3['dseta_step'], value=0, continuous_update=False)); # Plot results from simulation

### Save data

In [ ]:
data_ring3 =    {
                    'parameters_ring': parameters_ring3,
                    'simulation_options': simulation_options_ring3,
                    'forward_parameters': forward_parameters_ring3,
                    'dseta_forward': dseta_forward_ring3,
                    'amu_forward': amu_forward_ring3,
                }

In [ ]:
!mkdir -p data
with open('./data/data_ring3.pickle', 'wb') as f:
    pickle.dump(data_ring3, f, pickle.HIGHEST_PROTOCOL)

### Load data

In [ ]:
with open('./data/data_ring3.pickle', 'rb') as f:
    data = pickle.load(f)
    parameters_ring3 = data["parameters_ring"]
    simulation_options_ring3 = data["simulation_options"]
    forward_parameters_ring3 = data["forward_parameters"]
    dseta_forward_ring3 = data["dseta_forward"]
    amu_forward_ring3 = data["amu_forward"]

ring3 = rng.Ring(parameters_ring3) # Init Ring class

In [ ]:
interact(lambda dseta_snap: ring3.plot_results(dseta_forward_ring3, amu_forward_ring3, dseta_snap=dseta_snap), 
         dseta_snap=widgets.FloatSlider(min=forward_parameters_ring3['dseta_start'], max=forward_parameters_ring3['dseta_end'], step=forward_parameters_ring3['dseta_step'], value=0, continuous_update=False, font_size=75)); # Plot results from simulation